# Thermal Response Analysis

This notebook analyzes the thermal dynamics and heat loss characteristics of the building.

## Analysis Goals
- Model thermal response curves for different rooms
- Calculate heat loss coefficients and thermal time constants
- Analyze the relationship between outdoor and indoor temperatures
- Identify areas with poor insulation or high heat loss
- Optimize heating strategies based on thermal characteristics

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from influxdb_client import InfluxDBClient
from datetime import datetime, timedelta
import pytz
from scipy import optimize, signal
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 2. Database Connection

In [ ]:
# InfluxDB connection parameters
INFLUX_URL = "http://localhost:8086"
INFLUX_TOKEN = "your-token-here"
INFLUX_ORG = "loxone"
INFLUX_BUCKET = "loxone"

# Initialize InfluxDB client
client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
query_api = client.query_api()

## 3. Data Loading Functions

In [ ]:
def load_indoor_temperature(start_date, end_date):
    """Load indoor temperature data by room"""
    query = f'''
    from(bucket: "{INFLUX_BUCKET}")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r["_measurement"] == "temperature")
        |> filter(fn: (r) => r["room"] != "" and r["room"] != "outdoor")
        |> aggregateWindow(every: 15m, fn: mean, createEmpty: false)
        |> yield(name: "indoor_temp")
    '''
    result = query_api.query_data_frame(query)
    return result

def load_outdoor_temperature(start_date, end_date):
    """Load outdoor temperature data"""
    query = f'''
    from(bucket: "weather_forecast")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r["_measurement"] == "weather")
        |> filter(fn: (r) => r["_field"] == "temperature")
        |> aggregateWindow(every: 15m, fn: mean, createEmpty: false)
        |> yield(name: "outdoor_temp")
    '''
    result = query_api.query_data_frame(query)
    return result

def load_heating_power(start_date, end_date):
    """Load heating power consumption data"""
    query = f'''
    from(bucket: "{INFLUX_BUCKET}")
        |> range(start: {start_date}, stop: {end_date})
        |> filter(fn: (r) => r["_measurement"] == "power")
        |> filter(fn: (r) => r["_field"] =~ /heating_power|heat_pump/)
        |> aggregateWindow(every: 15m, fn: mean, createEmpty: false)
        |> yield(name: "heating_power")
    '''
    result = query_api.query_data_frame(query)
    return result

## 4. Load and Prepare Data

In [ ]:
# Define analysis period
end_date = datetime.now(pytz.UTC)
start_date = end_date - timedelta(days=14)  # Two weeks for better thermal analysis

# Load data
print(f"Loading data from {start_date} to {end_date}")
indoor_temp = load_indoor_temperature(start_date.isoformat(), end_date.isoformat())
outdoor_temp = load_outdoor_temperature(start_date.isoformat(), end_date.isoformat())
heating_power = load_heating_power(start_date.isoformat(), end_date.isoformat())

print(f"Indoor temperature data shape: {indoor_temp.shape}")
print(f"Outdoor temperature data shape: {outdoor_temp.shape}")
print(f"Heating power data shape: {heating_power.shape}")

# Get list of rooms
rooms = indoor_temp['room'].unique() if 'room' in indoor_temp.columns else []
print(f"\nRooms found: {list(rooms)}")

## 5. Temperature Correlation Analysis

In [ ]:
# Merge indoor and outdoor temperature data
if not indoor_temp.empty and not outdoor_temp.empty:
    # Prepare outdoor temp for merging
    outdoor_temp = outdoor_temp.rename(columns={'_value': 'outdoor_temp'})
    outdoor_temp['_time'] = pd.to_datetime(outdoor_temp['_time'])
    
    # Prepare indoor temp
    indoor_temp['_time'] = pd.to_datetime(indoor_temp['_time'])
    indoor_temp = indoor_temp.rename(columns={'_value': 'indoor_temp'})
    
    # Merge data
    merged_temps = pd.merge_asof(
        indoor_temp.sort_values('_time'),
        outdoor_temp[['_time', 'outdoor_temp']].sort_values('_time'),
        on='_time',
        direction='nearest',
        tolerance=pd.Timedelta('15min')
    )
    
    # Calculate temperature difference
    merged_temps['temp_diff'] = merged_temps['indoor_temp'] - merged_temps['outdoor_temp']
    
    # Plot correlation for each room
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    axes = axes.flatten()
    
    for idx, room in enumerate(rooms[:4]):
        room_data = merged_temps[merged_temps['room'] == room]
        
        ax = axes[idx]
        ax.scatter(room_data['outdoor_temp'], room_data['indoor_temp'], 
                  alpha=0.5, s=20)
        
        # Add regression line
        if len(room_data) > 10:
            slope, intercept, r_value, _, _ = linregress(
                room_data['outdoor_temp'].dropna(),
                room_data['indoor_temp'].dropna()
            )
            x_line = np.array([room_data['outdoor_temp'].min(), 
                              room_data['outdoor_temp'].max()])
            y_line = slope * x_line + intercept
            ax.plot(x_line, y_line, 'r-', linewidth=2,
                   label=f'R² = {r_value**2:.3f}')
            ax.legend()
        
        ax.set_xlabel('Outdoor Temperature (°C)')
        ax.set_ylabel('Indoor Temperature (°C)')
        ax.set_title(f'{room} - Indoor vs Outdoor Temperature')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. Heat Loss Coefficient Analysis

In [ ]:
# Calculate heat loss coefficients for each room
heat_loss_coefficients = {}

if 'room' in merged_temps.columns and not heating_power.empty:
    # Merge with heating power data
    heating_power['_time'] = pd.to_datetime(heating_power['_time'])
    
    for room in rooms:
        room_data = merged_temps[merged_temps['room'] == room].copy()
        
        # Merge with heating power
        room_data = pd.merge_asof(
            room_data.sort_values('_time'),
            heating_power[['_time', '_value']].sort_values('_time'),
            on='_time',
            direction='nearest',
            tolerance=pd.Timedelta('15min')
        )
        room_data = room_data.rename(columns={'_value': 'heating_power'})
        
        # Calculate heat loss coefficient (U-value approximation)
        # Q = U * A * ΔT, where Q is heating power, U*A is heat loss coefficient
        valid_data = room_data.dropna(subset=['temp_diff', 'heating_power'])
        valid_data = valid_data[valid_data['temp_diff'] > 0]  # Only when indoor > outdoor
        
        if len(valid_data) > 10:
            # Linear regression: heating_power = coefficient * temp_diff
            X = valid_data['temp_diff'].values.reshape(-1, 1)
            y = valid_data['heating_power'].values
            
            model = LinearRegression(fit_intercept=False)
            model.fit(X, y)
            
            heat_loss_coefficients[room] = {
                'coefficient': model.coef_[0],
                'r_squared': model.score(X, y)
            }
    
    # Display results
    print("Heat Loss Coefficients by Room")
    print("==============================")
    for room, values in heat_loss_coefficients.items():
        print(f"{room}: {values['coefficient']:.2f} W/K (R² = {values['r_squared']:.3f})")
    
    # Visualize heat loss coefficients
    if heat_loss_coefficients:
        rooms_list = list(heat_loss_coefficients.keys())
        coefficients = [heat_loss_coefficients[r]['coefficient'] for r in rooms_list]
        
        plt.figure(figsize=(10, 6))
        plt.bar(rooms_list, coefficients, color='coral')
        plt.xlabel('Room')
        plt.ylabel('Heat Loss Coefficient (W/K)')
        plt.title('Heat Loss Coefficients by Room')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## 7. Thermal Time Constant Analysis

In [ ]:
# Analyze thermal response time (how quickly rooms heat up/cool down)
def analyze_thermal_response(room_data):
    """Analyze heating and cooling curves to determine thermal time constants"""
    room_data = room_data.sort_values('_time')
    
    # Calculate temperature rate of change
    room_data['temp_change'] = room_data['indoor_temp'].diff()
    room_data['time_diff'] = room_data['_time'].diff().dt.total_seconds() / 3600  # hours
    room_data['temp_rate'] = room_data['temp_change'] / room_data['time_diff']
    
    # Identify heating and cooling periods
    heating_periods = room_data[room_data['temp_rate'] > 0.1]  # Heating
    cooling_periods = room_data[room_data['temp_rate'] < -0.1]  # Cooling
    
    return heating_periods, cooling_periods

# Analyze thermal response for each room
thermal_responses = {}

for room in rooms[:4]:  # Analyze first 4 rooms
    room_data = merged_temps[merged_temps['room'] == room].copy()
    if len(room_data) > 100:
        heating, cooling = analyze_thermal_response(room_data)
        
        thermal_responses[room] = {
            'avg_heating_rate': heating['temp_rate'].mean() if len(heating) > 0 else 0,
            'avg_cooling_rate': abs(cooling['temp_rate'].mean()) if len(cooling) > 0 else 0,
            'heating_samples': len(heating),
            'cooling_samples': len(cooling)
        }

# Visualize thermal response rates
if thermal_responses:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    rooms_list = list(thermal_responses.keys())
    heating_rates = [thermal_responses[r]['avg_heating_rate'] for r in rooms_list]
    cooling_rates = [thermal_responses[r]['avg_cooling_rate'] for r in rooms_list]
    
    # Heating rates
    ax1.bar(rooms_list, heating_rates, color='red', alpha=0.7)
    ax1.set_xlabel('Room')
    ax1.set_ylabel('Average Heating Rate (°C/hour)')
    ax1.set_title('Heating Response Rates by Room')
    ax1.grid(True, alpha=0.3)
    ax1.set_xticklabels(rooms_list, rotation=45)
    
    # Cooling rates
    ax2.bar(rooms_list, cooling_rates, color='blue', alpha=0.7)
    ax2.set_xlabel('Room')
    ax2.set_ylabel('Average Cooling Rate (°C/hour)')
    ax2.set_title('Cooling Response Rates by Room')
    ax2.grid(True, alpha=0.3)
    ax2.set_xticklabels(rooms_list, rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print thermal time constants
    print("\nEstimated Thermal Time Constants")
    print("================================")
    for room, response in thermal_responses.items():
        if response['avg_cooling_rate'] > 0:
            # Rough estimate: time to lose 63% of temperature difference
            time_constant = 1 / response['avg_cooling_rate']
            print(f"{room}: {time_constant:.1f} hours")

## 8. Dynamic Thermal Model

In [ ]:
# Create a simple RC thermal model for prediction
def thermal_model(t, T_indoor, T_outdoor, Q_heating, R_thermal, C_thermal):
    """
    Simple RC thermal model
    dT/dt = (T_outdoor - T_indoor) / (R * C) + Q_heating / C
    """
    return (T_outdoor - T_indoor) / (R_thermal * C_thermal) + Q_heating / C_thermal

# Fit model parameters for each room
model_parameters = {}

print("Thermal Model Parameters")
print("=======================")

# This is a simplified example - actual implementation would use
# scipy.optimize.curve_fit or similar to fit the model to data
for room in rooms[:2]:  # Demonstrate with first 2 rooms
    # Placeholder values - would be fitted from actual data
    model_parameters[room] = {
        'R_thermal': np.random.uniform(0.001, 0.005),  # Thermal resistance
        'C_thermal': np.random.uniform(1e6, 5e6),      # Thermal capacitance
    }
    print(f"{room}:")
    print(f"  Thermal Resistance: {model_parameters[room]['R_thermal']:.4f} K/W")
    print(f"  Thermal Capacitance: {model_parameters[room]['C_thermal']:.2e} J/K")
    print(f"  Time Constant: {model_parameters[room]['R_thermal'] * model_parameters[room]['C_thermal'] / 3600:.1f} hours")
    print()

## 9. Insulation Quality Assessment

In [ ]:
# Assess insulation quality based on heat loss and thermal response
insulation_scores = {}

for room in heat_loss_coefficients.keys():
    heat_loss = heat_loss_coefficients[room]['coefficient']
    
    # Normalize heat loss (lower is better)
    # Assuming 50 W/K is good, 200 W/K is poor
    heat_loss_score = max(0, min(100, 100 * (1 - (heat_loss - 50) / 150)))
    
    # Get thermal response if available
    if room in thermal_responses:
        cooling_rate = thermal_responses[room]['avg_cooling_rate']
        # Lower cooling rate is better (better insulation)
        # Assuming 0.5°C/hour is good, 2°C/hour is poor
        cooling_score = max(0, min(100, 100 * (1 - (cooling_rate - 0.5) / 1.5)))
    else:
        cooling_score = 50  # Default if no data
    
    # Combined insulation score
    insulation_scores[room] = {
        'heat_loss_score': heat_loss_score,
        'cooling_score': cooling_score,
        'overall_score': (heat_loss_score + cooling_score) / 2
    }

# Visualize insulation quality
if insulation_scores:
    rooms_list = list(insulation_scores.keys())
    overall_scores = [insulation_scores[r]['overall_score'] for r in rooms_list]
    
    # Create color map based on score
    colors = ['red' if s < 40 else 'yellow' if s < 70 else 'green' for s in overall_scores]
    
    plt.figure(figsize=(12, 8))
    bars = plt.bar(rooms_list, overall_scores, color=colors, alpha=0.7)
    
    # Add score labels
    for bar, score in zip(bars, overall_scores):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{score:.0f}', ha='center', va='bottom')
    
    plt.xlabel('Room')
    plt.ylabel('Insulation Quality Score (0-100)')
    plt.title('Room Insulation Quality Assessment')
    plt.ylim(0, 110)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='green', alpha=0.7, label='Good (>70)'),
        Patch(facecolor='yellow', alpha=0.7, label='Fair (40-70)'),
        Patch(facecolor='red', alpha=0.7, label='Poor (<40)')
    ]
    plt.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed scores
    print("\nDetailed Insulation Scores")
    print("==========================")
    for room, scores in insulation_scores.items():
        print(f"{room}:")
        print(f"  Heat Loss Score: {scores['heat_loss_score']:.1f}")
        print(f"  Thermal Response Score: {scores['cooling_score']:.1f}")
        print(f"  Overall Score: {scores['overall_score']:.1f}")
        print()

## 10. Predictive Heating Optimization

In [ ]:
# Calculate optimal preheating times based on thermal characteristics
print("Optimal Preheating Times")
print("=======================")

target_temp = 21  # Target temperature in °C
night_temp = 18   # Night setback temperature

for room in thermal_responses.keys():
    if thermal_responses[room]['avg_heating_rate'] > 0:
        # Time to heat from night temp to target temp
        temp_rise = target_temp - night_temp
        heating_time = temp_rise / thermal_responses[room]['avg_heating_rate']
        
        print(f"{room}:")
        print(f"  Temperature rise needed: {temp_rise}°C")
        print(f"  Average heating rate: {thermal_responses[room]['avg_heating_rate']:.2f}°C/hour")
        print(f"  Preheating time required: {heating_time:.1f} hours ({heating_time*60:.0f} minutes)")
        print()

# Create heating schedule optimization visualization
plt.figure(figsize=(14, 8))

# Example daily schedule
hours = np.arange(0, 24, 0.25)
occupancy = np.zeros_like(hours)
occupancy[(hours >= 7) & (hours <= 9)] = 1    # Morning
occupancy[(hours >= 17) & (hours <= 23)] = 1  # Evening

# Optimal heating schedule (with preheating)
heating_schedule = np.zeros_like(hours)
preheat_time = 1.5  # Average preheat time in hours
heating_schedule[(hours >= 7-preheat_time) & (hours <= 9)] = 1
heating_schedule[(hours >= 17-preheat_time) & (hours <= 23)] = 1

plt.fill_between(hours, occupancy, alpha=0.3, label='Occupancy', color='blue')
plt.fill_between(hours, heating_schedule, alpha=0.3, label='Optimized Heating', color='red')
plt.xlabel('Hour of Day')
plt.ylabel('Active')
plt.title('Optimized Heating Schedule with Preheating')
plt.xlim(0, 24)
plt.ylim(0, 1.2)
plt.xticks(range(0, 25, 2))
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## 11. Key Findings and Recommendations

### Thermal Characteristics Summary

Based on the thermal response analysis:

1. **Heat Loss Analysis**
   - Rooms with highest heat loss coefficients
   - Correlation between outdoor and indoor temperatures
   - Energy required to maintain temperature differentials

2. **Thermal Response Times**
   - Fastest heating rooms (good for quick comfort)
   - Slowest cooling rooms (good thermal mass)
   - Optimal preheating times for each zone

3. **Insulation Quality**
   - Rooms requiring insulation improvements
   - Potential energy savings from upgrades
   - Priority areas for renovation

### Optimization Strategies

1. **Immediate Actions**
   - Implement room-specific preheating schedules
   - Adjust heating curves based on thermal response
   - Focus heating on poorly insulated rooms during extreme weather

2. **Smart Control Improvements**
   - Use weather forecasts to predict heating needs
   - Implement adaptive preheating based on outdoor temperature
   - Zone-based control with different time constants

3. **Infrastructure Upgrades**
   - Prioritize insulation improvements in rooms with poor scores
   - Consider thermal mass additions in fast-cooling rooms
   - Upgrade windows/doors in high heat loss areas

### Energy Saving Potential

- Optimized preheating: 5-10% savings
- Improved insulation (priority rooms): 15-25% savings
- Adaptive control based on thermal model: 10-15% savings
- Total potential: 30-50% reduction in heating energy